In [34]:
from sssom.parsers import parse_sssom_table
import pandas as pd
mondo_mappings = "/Users/matentzn/ws/mondo/src/ontology/mappings/mondo.sssom.tsv" #http://purl.obolibrary.org/obo/mondo/mappings/mondo.sssom.tsv
msdf = parse_sssom_table(mondo_mappings)

In [35]:
df = msdf.df
df.head()

,subject_id,subject_label,predicate_id,object_id,object_label,mapping_justification
0,MONDO:0000001,disease,skos:exactMatch,DOID:4,,semapv:UnspecifiedMatching
1,MONDO:0000001,disease,skos:exactMatch,MESH:D004194,,semapv:UnspecifiedMatching
2,MONDO:0000001,disease,skos:exactMatch,NCIT:C2991,Disease or Disorder,semapv:UnspecifiedMatching
3,MONDO:0000001,disease,skos:exactMatch,Orphanet:377788,,semapv:UnspecifiedMatching
4,MONDO:0000001,disease,skos:exactMatch,SCTID:64572001,,semapv:UnspecifiedMatching


In [36]:
df['object_prefix'] = df.apply (lambda x: x['object_id'].split(":")[0], axis=1)
df_prefix_per_class = df.groupby(['subject_id', 'object_prefix'])['object_prefix'].count().reset_index(name='count')
df_more_than_one_prefix_per_class = df_prefix_per_class[df_prefix_per_class['count']>1]
prefix_proxy_merges = df_more_than_one_prefix_per_class.groupby(['object_prefix'])['object_prefix'].count().reset_index(name='count')
prefix_proxy_merges.head(10)

,object_prefix,count
0,DOID,114
1,ICD10CM,4
2,MESH,68
3,NCIT,39
4,OMIM,7
5,Orphanet,34
6,SCTID,79
7,UMLS,993


In [46]:
df_prefix_per_class[(df_prefix_per_class['object_prefix']!='UMLS') & (df_prefix_per_class['count']>2)]

,subject_id,object_prefix,count
13924,MONDO:0004992,DOID,3
14556,MONDO:0005165,DOID,3
18831,MONDO:0006639,DOID,3
38652,MONDO:0011781,MESH,3
40264,MONDO:0012215,MESH,3
40266,MONDO:0012215,Orphanet,3
40267,MONDO:0012215,SCTID,3
45177,MONDO:0013626,Orphanet,3
51517,MONDO:0015912,SCTID,4
59000,MONDO:0019046,DOID,3


In [38]:
df_more_than_one_prefix_per_class.head()

,subject_id,object_prefix,count
68,MONDO:0000107,UMLS,2
188,MONDO:0000212,SCTID,2
271,MONDO:0000261,UMLS,2
282,MONDO:0000265,UMLS,2
296,MONDO:0000276,UMLS,2


In [39]:
len(df_more_than_one_prefix_per_class)

1338

In [40]:
import pandas as pd
df_proxy_merges=pd.merge(df,df_more_than_one_prefix_per_class, left_on=['subject_id','object_prefix'], right_on=['subject_id','object_prefix'])
df_proxy_merges.head()
df_proxy_merges.to_csv('proxymerges-mondo.sssom.tsv',sep="\t", index=False)
